In [ ]:
import backtrader as bt
from datetime import datetime
import pandas as pd

# Load data from CSV
df = pd.read_csv('./MSFT.csv', index_col='Date', parse_dates=True)
df = df[df.index >= '2025-01-01']
df = df[df.index < '2025-10-01']
pandasdata = bt.feeds.PandasData(dataname=df, open=0, high=1, low=2, close=3, volume=5)
print("Data loaded successfully.")

Data loaded successfully.


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT at the first day start_date. 

2. Hold the 100 shares until end_date without buying additional shares. 

3. Output the return of this strategy.

In [ ]:
class TestStrategy(bt.Strategy):
    # Put your strategy below.
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.data.close

    def next(self):
        if (len(self)==1):
            self.buy(size=100)


In [ ]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


The one year return of the strategy is: 0.09460000610351563
The cash left is: 57664.99938964844
Current position: 100 shares of MSFT


---

1. Backtest a strategy with backtrader that ALL-IN MSFT from the first day start_date. 

2. Hold all the shares until end_date. 

3. Output the return of this strategy.

In [ ]:
class TestStrategy(bt.Strategy):
    # Put your strategy below.

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.data.close
        
    def next(self):
        # The self.order_target_percent() method in Backtrader is a convenient way to execute 
        # trades based on a target percentage of the available cash.
        self.order_target_percent(target=1.0)

In [ ]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Bought 235 shares at $425.53 with $100000.00 cash
The one year return of the strategy is: 0.2276445602416992
The cash left is: 1046.2031555175781
Current position: 235 shares of MSFT


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT every 30 trading days until out of cash from the first day start_date. 

2. For example, the first buying date is the 1st trading day since start_date, the second buying date is the 31th trading day, the third is 61 trading date, etc. 

3. Hold all the shares until end_date. 

4. Output the return of this strategy.

In [ ]:
class TestStrategy(bt.Strategy):
    # Put your strategy below.
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.data.close
        
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                # Buy order was filled
                filled_price = order.executed.price
                self.log(f"Buy order filled at price: {filled_price}")

    def next(self):
        if (len(self)%30==1):
            self.buy(size=100)
        

In [ ]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Day 1: Bought 100 shares at $418.58, Cost: $41858.00, Cash left: $58142.00
Day 31: Bought 100 shares at $409.64, Cost: $40964.00, Cash left: $16928.00
Day 61: Not enough cash to buy 100 shares. Need $38219.00, Have $17104.00
Day 91: Not enough cash to buy 100 shares. Need $45294.00, Have $17104.00
Day 121: Not enough cash to buy 100 shares. Need $49594.00, Have $17104.00
Day 151: Not enough cash to buy 100 shares. Need $52177.00, Have $17104.00
Day 181: Not enough cash to buy 100 shares. Need $50923.00, Have $17104.00
The one year return of the strategy is: 0.2069400329589844
The cash left is: 17104.000854492188
Current position: 200 shares of MSFT


---

1. Backtest a strategy with backtrader that buys 100 shares of MSFT when the loss of a day is over 2\% (compare the close price with open price in one day)

In [ ]:
class TestStrategy(bt.Strategy):
    # Put your strategy below.
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

        
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                # Buy order was filled
                filled_price = order.executed.price
                self.log(f"Buy order filled at price: {filled_price}")

    def next(self):
        close_price,open_price = self.data.close[0], self.data.open[0]
        ret = (close_price-open_price)/open_price
        if (ret < -0.02):
            self.buy(size=100)    
    

In [ ]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    init_cash = 100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(TestStrategy)
    cerebro.adddata(pandasdata, name='MSFT')
    cerebro.run()
    
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(pandasdata)
    ret = (value-init_cash)/init_cash
    print (f'The one year return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')


Day 34: Daily loss -2.19% (>2%), bought 100 shares at $408.21
Day 38: Daily loss -2.18% (>2%), bought 100 shares at $392.53
Day 40: Not enough cash to buy 100 shares. Need $38849.00, Have $19901.00
Day 59: Not enough cash to buy 100 shares. Need $37880.00, Have $19901.00
Day 66: Not enough cash to buy 100 shares. Need $35456.00, Have $19901.00
Day 72: Not enough cash to buy 100 shares. Need $37161.00, Have $19901.00
Day 144: Not enough cash to buy 100 shares. Need $53350.00, Have $19901.00
Day 145: Not enough cash to buy 100 shares. Need $52411.00, Have $19901.00
Day 153: Not enough cash to buy 100 shares. Need $52058.00, Have $19901.00
Day 169: Not enough cash to buy 100 shares. Need $49500.00, Have $19901.00
The one year return of the strategy is: 0.2349100341796875
The cash left is: 19901.0009765625
Current position: 200 shares of MSFT
